# 商业数据分析实战（酒卷隆志/里洋平）——第三章 案例 A—柱状图:为什么销售额会减少
　　**案例提要：社交游戏的销售额分析**  
　　一款叫做《黑猫拼图》的社交游戏本月的销售额相较于上月有所下滑，于是想调查下滑的原因，并提升销售额，该怎么做呢？

## 一、现状与预期
　　![](《黑猫拼图》的上月与本月的销售额比较1.png)
　　**现状**：从上图可以看出，和上月相比本月的销售额确实下降了。  
　　**预期**：而无论是从市场环境还是游戏本身的状态来看，这个游戏的销售额还有继续增长的空间。也就是说，本来的预期是“希望能够确
保和上月相同的销售额”。  
　　![](第3章的现状和预期.png)

## 二、发现问题
　　**目的**：解析现状与预期差距的构造，明确现状与预期之间的差距在哪？  
　　**方法**：比较本月与上月有哪些地方不同？  
　　**具体步骤：**  
　　**提出假设**：在最初阶段，重要的是从大局出发来考量，而不是局限于数据分析的范畴。例如，可以根据业务经验进行下述假设：  
　　作为可能导致销售额减少的原因，上月和本月的不同之处有以下两点。  
　　1、在商业宣传上存在问题  
　　2、每月以不同的主题开展的游戏活动存在问题  
　　**验证假设**：通过咨询市场部和游戏开发部，得到了以下信息：  
　　1、由于预算的缘故，和上月相比，本月并没有开展那么多的商业宣传活动。  
　　2、游戏活动的内容和上月相比几乎没有变动。  
　　根据第 2 条的内容可知，游戏活动的内容相比之前没有大的变化，那么原因很可能是第 1 条假设——“商业宣传力度不够导致了销售额下降”。然后再根据业务知识进行下一步的假设：由于商业宣传活动减少了，因此就很难有更多的人了解公司产品，产品也就很难获得新的用户，新用户的减少导致了销售额的下降。  
　　**假设小结**：  
　　● 和上月相比，本月的销售额减少了  （事实）  
　　● 本月的商业宣传活动相比上月减少了 （事实）  
　　● 可能是新用户的数量的减少引起 本月的销售额减少（假设）  
　　● 将商业宣传活动恢复到与上月相同的水平（解决方案）  
　　为了验证假设是否正确，需要通过收集和加工数据进行分析《黑猫拼图》游戏销售额的构成。

## 三、数据的收集与加工  
　　**1、收集所需的数据**  
　　主要收集这3个日志数据：  
　　● DAU（Daily Active User，每天至少来访 1 次的用户数据）（用户访问数据） 

| 数据内容 | 字段英文名 | 数据类型 | 
| - | :-: | -: | 
| 访问时间 | log_data| string | 
| 应用名称 | app_name | string |
| 用户 ID | user_id | int|

　　● DPU（Daily Payment User，每天至少消费 1 日元的用户数据）（用户消费数据） 

| 数据内容 | 字段英文名 | 数据类型 | 
| - | :-: | -: | 
| 消费日期 | log_data| string | 
| 应用名称 | app_name | string |
| 用户 ID | user_id | int|
| 消费额 | Payment | int |

　　● Install（记录每个用户首次玩这个游戏的时间的数据）  

| 数据内容 | 字段英文名 | 数据类型 | 
| - | :-: | -: | 
| 首次使用日期 | log_data| string | 
| 应用名称 | app_name | string |
| 用户 ID | user_id | int|


　　使用python导入数据代码如下：

In [16]:
# 导工具包
import pandas as pd
import numpy as np
import matplotlib as plt
# 收集分析所需的数据
dau = pd.read_csv("../data/section3-dau.csv")
dau.head(5)

,log_date,app_name,user_id
0,2013-06-01,game-01,116
1,2013-06-01,game-01,13491
2,2013-06-01,game-01,7006
3,2013-06-01,game-01,117
4,2013-06-01,game-01,13492


In [18]:
dpu = pd.read_csv("../data/section3-dpu.csv")
dpu.head(5)

,log_date,app_name,user_id,payment
0,2013-06-01,game-01,351,1333
1,2013-06-01,game-01,12796,81
2,2013-06-01,game-01,364,571
3,2013-06-01,game-01,13212,648
4,2013-06-01,game-01,13212,1142


In [17]:
install = pd.read_csv("../data/section3-install.csv")
install.head(5)

,install_date,app_name,user_id
0,2013-04-15,game-01,1
1,2013-04-15,game-01,2
2,2013-04-15,game-01,3
3,2013-04-15,game-01,4
4,2013-04-15,game-01,5


　　**2、加工分析所需的数据（前期处理）**  
　　在加工过程中我们要随时谨记我们的目的：判断“销售额减少是否受到了新用户因素的影响”。  
　　**（1）导入工具包以及数据 ** 

![](图片1.png)

In [ ]:
# 探索数据
print(dau.count())
print(dau.drop_duplicates(['user_id','app_name','log_date']).count())
print(dpu.count())
print(dpu.drop_duplicates(['user_id','app_name','log_date']).count())
print(install.count())
print(install.drop_duplicates(['user_id','app_name']).count())

In [ ]:
# 加工数据
# 合并数据
dau_dpu = dau.merge(dpu,how='left',on=['user_id','app_name','log_date'])
dau_dpu_install = dau_dpu.merge(install,how='left',on=['user_id','app_name'])
dau_dpu_install

In [50]:
# 将未消费用户的消费额设置为零
dau_dpu_install = dau_dpu_install.fillna(value={'payment':0})
dau_dpu_install.head(5)

,log_date,app_name,user_id,payment,install_date
0,2013-06-01,game-01,116,0.0,2013-04-17
1,2013-06-01,game-01,13491,0.0,2013-06-01
2,2013-06-01,game-01,7006,0.0,2013-05-03
3,2013-06-01,game-01,117,0.0,2013-04-17
4,2013-06-01,game-01,13492,0.0,2013-06-01


In [130]:
# 按月份进行统计消费数据,截取日期为月并进行统计,最终得到每个用户每月的消费额
# 不知道为什么pd.to_period()总是提示该函数不存在，可是我在官方文档还有help（）和源码里面能找到啊，求大佬解惑
#dau_dpu_install['log_month'] = pd.to_datetime(dau_dpu_install['log_date'],format="%Y-%m-%d").to_period("M")
dau_dpu_install['log_month'] = pd.to_datetime(dau_dpu_install['log_date'],format="%Y-%m-%d").dt.to_period("M")
dau_dpu_install['install_month'] = pd.to_datetime(dau_dpu_install['install_date'],format="%Y-%m-%d").dt.to_period("M")
mau_payment = dau_dpu_install.groupby(['user_id','log_month','install_month']).sum().reset_index()
mau_payment

,user_id,log_month,install_month,payment
0,1,2013-06,2013-04,0.0
1,1,2013-07,2013-04,0.0
2,2,2013-06,2013-04,0.0
3,3,2013-06,2013-04,14994.0
4,3,2013-07,2013-04,0.0
5,4,2013-06,2013-04,0.0
6,4,2013-07,2013-04,0.0
7,6,2013-06,2013-04,0.0
8,6,2013-07,2013-04,0.0
9,7,2013-06,2013-04,0.0


In [139]:
# 增加属性来区分新用户与已有用户，最终得到每个用户每月是否为新用户的消费额
# 核心在于如何判断新用户属性：设置时间段“月”，登录时间是否等于注册时间
case_when = lambda x: '新用户' if (x['log_month'] == x['install_month']) else '已有用户'
mau_payment['user_type'] = mau_payment.apply(case_when,axis=1)

In [157]:
# 将数据保存到excel进行画图
mau_payment.to_excel('mau_payment.xlsx',index=False)

In [143]:
# 计算每月已有用户和新用户的销售额
mau_payment_user_type = mau_payment.groupby(['log_month','user_type']).sum().loc[:,['payment']].reset_index()
mau_payment_user_type

,log_month,user_type,payment
0,2013-06,已有用户,177886.0
1,2013-06,新用户,49837.0
2,2013-07,已有用户,177886.0
3,2013-07,新用户,29199.0
